# Comparison between original model and model with delay

Acapacidade média de atendimento por passo em cada um dos modelos comparados deve ser a mesma. Para isso, o número máxmio de servidores no caso com atraso deve ser igual a $S_d$

$$S_d = t_m \cdot S_o$$

onde $t_m$ é o tempo médio de serviço (1+atraso), $S_o$ é o número de servidores no modelo original sem atrasos.

Para um atraso máximo no serviço de 3 passos ($t_{serM} = 3$), com atrasos distribuidos de maneira uniforme, o $t_m$ seria igual a 3. Para que a comparação seja válida, é necessário que $t_m$ seja um inteiro. Para que isso ocorra, $t_serM$ deve ser ímpar.

$$t_m = \frac{((t_{serM}+1)(t_{serM}+2)/2)-1}{t_{serM}} = \frac{2+3+4}{3} = 3 $$

In [1]:
using JuMP
using HiGHS
using Ipopt
using Juniper
using DelimitedFiles
using Serialization
using Random

include("utils.jl") 
include("optimize_list.jl")

optimize_list (generic function with 1 method)

### Exogenous variables

In [2]:
N = 10           # number of examples
horiz = 20       # total horizon

limit_count = 10 # max number of trials to create a new replacement demand in case of infeasibility

createDemands = false

d_prop = distribution_properties(10, 0, "uniform"); # properties of the demand distribution
a_prop = distribution_properties(1, 0, "uniform");

if createDemands
    d_mat = demand_generator_mat(N, horiz, d_prop.M, d_prop.type, d_prop.std_dev);
    a_mat = demand_generator_mat(N, horiz, a_prop.M, a_prop.type, a_prop.std_dev);
else
    d_fn = string("exogenous_vars//d_mat_", d_prop.type, "_max_", d_prop.M, "_dev_", d_prop.std_dev, ".txt");
    a_fn = string("exogenous_vars//a_mat_", a_prop.type, "_max_", a_prop.M, "_dev_", a_prop.std_dev, ".txt");
    d_mat = DelimitedFiles.readdlm(d_fn);
    a_mat = DelimitedFiles.readdlm(a_fn);
end
;

### Parametrers

In [3]:
compensateSerDelay = true
compensateCostDelay = false

# bounds
XM = 6            # max queue length 
YM = 10           # max buffer length before dropping calls
phiM = 9          # max adimission to queue
serM_nd = 5       # max number of servers original model
tserM = 3         # max delay in service time

# iniital conditions
X0 = 4
Y0 = 6
L0 = 0
Z0 = 1
tm = (((tserM+1)*(tserM+2)/2)-1)/tserM # average service time

# Non-linear objective function costs
c_blr_nd = 400
c_ser_nd = 1

# Linear objective function costs
if compensateCostDelay
    c_Z_d = 3/2/tm      # weight of served clients      
    c_ser_d = 1/tm      # weight of server cost
    c_L_d = 1/2/tm      # weight of buffer cost
else
    c_Z_d = 3/2               
    c_ser_d = 1         
    c_L_d = 1/2         
end

# Adjust max number of servers for the model with delay
if compensateSerDelay
    serM_d = Int(serM_nd*tm)                  
else
    serM_d = serM_nd                        
end

bds_nd = bounds(XM, YM, phiM, serM_nd, 0)
bds_d = bounds(XM, YM, phiM, serM_d, tserM)

    
ic = initial_conditions(X0, Y0, L0, Z0)

c_nd = c(0, 0, c_ser_nd, c_blr_nd)  # original model costs
c_d = c(c_Z_d, c_L_d, c_ser_d, c_blr_nd)  # model with delay costs

df_input = df_input_generator(horiz, bds_d.serM, bds_d.tserM);

### Model without delay

In [5]:
result_os_nd, d_mat_os_nd, a_mat_os_nd, count_os_nd = optimize_list("nd_os", d_mat, a_mat);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
result_om_nd, d_mat_om_nd, a_mat_om_nd, count_om_nd = optimize_list("nd_om", d_mat, a_mat);

### Model with delay

In [4]:
result_os_d, d_mat_os_d, a_mat_os_d, count_os_d = optimize_list("d_os", d_mat, a_mat);

INFEASIBLE
At least one solution is infeasible, replacing demand 3(count = 1/10)
INFEASIBLE
At least one solution is infeasible, replacing demand 4(count = 1/10)
INFEASIBLE
At least one solution is infeasible, replacing demand 4(count = 2/10)
Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [7]:
result_os_d_nonlin, d_mat_os_d_nonlin, a_mat_os_d_nonlin, count_os_d_nonlin = optimize_list("d_os_nonlin", d_mat, a_mat);

Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [17]:
# result_om_d, d_mat_om_d, a_mat_om_d, count_om_d = optimize_list("d_om", d_mat, a_mat);

In [20]:
result_om_d_nonlin, d_mat_om_d_nonlin, a_mat_om_d_nonlin, count_om_d_nonlin = optimize_list("d_om_nonlin", d_mat, a_mat);

[0.0, -0.0, 0.058823529411764705, 0.05555555555555555, 0.08, 0.1, 0.08333333333333333, 0.0975609756097561, 0.09302325581395349, 0.10204081632653061, 0.08928571428571429][-0.0, 0.3125, 0.23809523809523808, 0.5142857142857142, 0.4, 0.42592592592592593, 0.47692307692307695, 0.44285714285714284, 0.43037974683544306, 0.42696629213483145, 0.4329896907216495][-0.0, 0.08333333333333333, 0.2727272727272727, 0.39285714285714285, 0.3157894736842105, 0.3541666666666667, 0.3333333333333333, 0.3333333333333333, 0.3382352941176471, 0.34615384615384615, 0.35294117647058826][-0.0, -0.0, 0.15789473684210525, 0.19047619047619047, 0.16129032258064516, 0.16216216216216217, 0.15, 0.23076923076923078, 0.27419354838709675, 0.2608695652173913, 0.2465753424657534]

┌ Warning: MIP couldn't be solved to optimality. Terminated with status: TIME_LIMIT
└ @ Juniper ~/.julia/packages/Juniper/yEX2j/src/fpump.jl:365


[0.0, 0.3125, 0.23809523809523808, 0.37037037037037035, 0.35714285714285715, 0.3404255319148936, 0.3333333333333333, 0.3157894736842105, 0.296875, 0.2916666666666667, 0.34523809523809523][-0.0, 0.09090909090909091, 0.058823529411764705, 0.22727272727272727, 0.1935483870967742, 0.24390243902439024, 0.24444444444444444, 0.2857142857142857, 0.3283582089552239, 0.37037037037037035, 0.40217391304347827][0.0, 0.26666666666666666, 0.4074074074074074, 0.39285714285714285, 0.3076923076923077, 0.29545454545454547, 0.2916666666666667, 0.2777777777777778, 0.25, 0.22727272727272727, 0.21428571428571427][0.0, 0.09090909090909091, 0.21052631578947367, 0.2916666666666667, 0.23529411764705882, 0.225, 0.22727272727272727, 0.24528301886792453, 0.3076923076923077, 0.28169014084507044, 0.26666666666666666]

┌ Warning: MIP couldn't be solved to optimality. Terminated with status: TIME_LIMIT
└ @ Juniper ~/.julia/packages/Juniper/yEX2j/src/fpump.jl:365


[0.0, 0.23076923076923078, 0.30434782608695654, 0.32, 0.2972972972972973, 0.29545454545454547, 0.2916666666666667, 0.2857142857142857, 0.26229508196721313, 0.23880597014925373, 0.2361111111111111][-0.0, 0.15384615384615385, 0.15789473684210525, 0.45161290322580644, 0.35714285714285715, 0.32608695652173914, 0.30612244897959184, 0.2909090909090909, 0.26229508196721313, 0.23880597014925373, 0.2361111111111111]Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

## Results comparison

In [ ]:
# results_list = [result_os_nd, result_om_nd, result_os_d, result_om_d]
results_list = [result_os_nd, result_os_d, result_os_d_nonlin, result_om_d_nonlin]

a_mat_list = [a_mat_os_nd, a_mat_os_nd, a_mat_os_d, a_mat_os_d_nonlin, a_mat_os_d]
d_mat_list = [d_mat_os_nd, d_mat_os_nd, d_mat_os_d, d_mat_os_d_nonlin, d_mat_os_d]
count_list = [count_os_nd, count_os_nd, count_os_d, count_os_d_nonlin, count_os_d]

res_stats = Dict()

for i in 1:length(results_list)
    res = results_list[i]
    result_id = res.id*"_"*string(horiz)*"_"*string(d_prop.M)*"_"*string(a_prop.M)*"_"*string(res.bds.XM)*"_"*string(res.bds.YM)*"_"*string(res.bds.phiM)*"_"*string(res.bds.serM)*"_"*string(res.bds.tserM)
    cost, av_blr, av_dr, av_ser, total_clients = compute_metrics(res, horiz, N)

    # Print results table
    println(res.id)
    data = hcat(av_blr, total_clients, av_dr, av_ser, cost)
    header = (["average blr", "Average served clients", "Average dropped clients", "Average active Servers", "Average cost function value"]);
    printTable(data, header)

    # # add csv line
    # add_line_to_csv(result_id, res, horiz, d_prop, a_prop, cost, av_blr, av_dr, av_ser, total_clients, count_list[i])

    # # Save results to file
    # var_dict = Dict("res"=>res, "a"=>a_mat_list[i], "d"=>d_mat_list[i])
    # fn = string("results//",result_id,".txt")
    # serialize(fn, var_dict)
end

nd_os
Results
┌─────────────┬────────────────────────┬─────────────────────────┬────────────────────────┬─────────────────────────────┐
│ average blr │ Average served clients │ Average dropped clients │ Average active Servers │ Average cost function value │
├─────────────┼────────────────────────┼─────────────────────────┼────────────────────────┼─────────────────────────────┤
│       0.332 │                 50.800 │                   2.170 │                  4.980 │                     132.798 │
└─────────────┴────────────────────────┴─────────────────────────┴────────────────────────┴─────────────────────────────┘
d_os
Results
┌─────────────┬────────────────────────┬─────────────────────────┬────────────────────────┬─────────────────────────────┐
│ average blr │ Average served clients │ Average dropped clients │ Average active Servers │ Average cost function value │
├─────────────┼────────────────────────┼─────────────────────────┼────────────────────────┼────────────────────────────

### Create demands

In [9]:
createDemands = true

N = 15            # number of examples
horiz = 50        # total horizon
 
M_list = [15]  # max demand
# M_list = [1,3]

std_dev_list = [0]

type_distr = "uniform"
exo_var = "d"

for M in M_list
    if type_distr == "uniform"
        std_dev_list = [0]
    end
    for std in std_dev_list
        d_mat = demand_generator_mat(N, horiz, M, type_distr, std)
        d_mat_fn = "exogenous_vars//"* exo_var*"_mat_"* type_distr* "_max_"* string(M)* "_dev_"* string(std)* ".txt"
        writedlm(d_mat_fn, d_mat)
    end
end


### Check results

In [91]:
function check_results(res, a, d)
    for i in 1:N
        if res.status_opt[i] == 0
            println("The optimization problem n ", i," did not solve to optimality.")
        else
            for t in 1:horiz
                @assert res.X[i,t+1] == res.X[i,t] + res.phi[i,t] - a[t,i] - res.Cin[i,t]
                @assert res.Y[i,t+1] == res.Y[i,t] + res.Q[i,t] - res.phi[i,t]
                @assert res.Z[i,t+1] == res.Z[i,t] + res.Cin[i,t]
                @assert res.L[i,t+1] == res.L[i,t] + res.dr[i,t] + a[t,i]
                @assert res.Cin[i,t] == sum(res.Sl[i,t,:])
            end
        end
    end
    println("All constraints are satisfied")a
end

check_results (generic function with 1 method)